In [55]:
# import library
!pip install nltk
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# download APIs for natural language processing
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk

True

In [56]:
# read data from CSV file
df = pd.read_csv('../data/raw/MarvelTheFantasticFourFirstSteps.csv')
# df = pd.read_csv('marvelRanarok.csv', encoding="cp1252")


review = df["review"].tolist()

# convert label from text to digit
df["sentiment"] = df["sentiment"].map({"positive" : 1, "negative" : -1, "neutral" : 0})
sentiment = df["sentiment"].tolist()

print(review)
print(sentiment)
# print(df.to_string())
# print(df.loc[2, "review"])
# print(df.info())
# print(df["sentiment"].value_counts())
# pd.options.display.max_rows = 9999
# print(pd.options.display.max_rows)
# csv_size = len(df)


# print(csv_size)
# print(X[0])
# print(y)


['The saddest thing about this new movie is the fact that Stan Lee and Jack Kirby, bless their hearts, are no longer around to see one of their most important creations, after years of mistreatment, finally done justice on the big screen.', 'Most important creations? Please stop it', 'It was not good', 'it was mediocre', 'No ones watching it after the first weekend lol. It was not good. Superman was ok...and better lol', 'im not hating. It was ok. Decent. Too jumpy...torch and things relationship was off...Pascal was miscast and they used a one off female surfer. Could have been so much better. Galactus looked terrible also', 'Amen to that!!! Marvel, I love you forever 3000!!! ❤❤❤', 'it was decent. A lot of flaws...but watchable.', 'Dude this trailer gives me goosebumps', 'Horrible false statement not even an opinion', 'Just saw this movie earlier today and omg it was so good!!!', 'I love how it’s a running joke that Ben simply refuses to say “ITS CLOBBERIN TIME”. Can’t wait to see thi

In [57]:
# create pipeline for processing data
# import word tokenizer for tokenization process and pos_tag for speech tagging 
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def preprocess_data(review):
    token = word_tokenize(review)
    tagged = pos_tag(token)
    # keep adjactive (JJ), adverb (RB) and verb (VB)
    important_tokens = [w for w, pos in tagged if pos.startswith("JJ") or pos.startswith("RB") or pos.startswith("VB") or pos.startswith("NN")]
    preprocessed_text = " ".join(token)
    return preprocessed_text

preprocessed_text = [preprocess_data(text) for text in review]
print(preprocessed_text)

# for i in range(csv_size):
#     print(i)
#     print("Before: " + df.loc[i, "review"])
#     data_Tokenized = word_tokenize(df.loc[i, "review"].lower())
#     print(data_Tokenized)
#     tagged_tokens = pos_tag(data_Tokenized)
#     print("Result: " + str(tagged_tokens))


# X = X.to_string()
# X_process = X.lower()

# print(X)
# print(word_tokenize(X))

['The saddest thing about this new movie is the fact that Stan Lee and Jack Kirby , bless their hearts , are no longer around to see one of their most important creations , after years of mistreatment , finally done justice on the big screen .', 'Most important creations ? Please stop it', 'It was not good', 'it was mediocre', 'No ones watching it after the first weekend lol . It was not good . Superman was ok ... and better lol', 'im not hating . It was ok . Decent . Too jumpy ... torch and things relationship was off ... Pascal was miscast and they used a one off female surfer . Could have been so much better . Galactus looked terrible also', 'Amen to that ! ! ! Marvel , I love you forever 3000 ! ! ! ❤❤❤', 'it was decent . A lot of flaws ... but watchable .', 'Dude this trailer gives me goosebumps', 'Horrible false statement not even an opinion', 'Just saw this movie earlier today and omg it was so good ! ! !', 'I love how it ’ s a running joke that Ben simply refuses to say “ ITS CL

In [58]:
# vectorization
# convert text to vector for SVM
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_text) 
y = df["sentiment"].tolist() # label
# print(vectorizer.get_feature_names_out())
print(X)
print(y)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2577 stored elements and shape (141, 888)>
  Coords	Values
  (0, 759)	0.17020251299436545
  (0, 637)	0.18671151030666297
  (0, 769)	0.1422654730463593
  (0, 19)	0.12961121342439935
  (0, 773)	0.06212746420974963
  (0, 521)	0.14773449394784102
  (0, 509)	0.06487910698677098
  (0, 418)	0.08224749926082085
  (0, 267)	0.18671151030666297
  (0, 758)	0.08319364624415986
  (0, 714)	0.18671151030666297
  (0, 448)	0.18671151030666297
  (0, 51)	0.07546916338605404
  (0, 421)	0.18671151030666297
  (0, 436)	0.1723262532740632
  (0, 110)	0.18671151030666297
  (0, 762)	0.3446525065481264
  (0, 367)	0.18671151030666297
  (0, 63)	0.13752799165421858
  (0, 524)	0.11767371372013712
  (0, 462)	0.18671151030666297
  (0, 64)	0.1723262532740632
  (0, 785)	0.08224749926082085
  (0, 652)	0.1262297598618335
  (0, 546)	0.1262297598618335
  :	:
  (140, 694)	0.12978954996642572
  (140, 319)	0.10303232720407042
  (140, 22)	0.2695579338179921
  (140, 41)

In [59]:
# split train data and test data and train SVM model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = SVC(kernel="linear").fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          -1       0.57      1.00      0.73        12
           0       0.00      0.00      0.00         5
           1       0.75      0.50      0.60        12

    accuracy                           0.62        29
   macro avg       0.44      0.50      0.44        29
weighted avg       0.55      0.62      0.55        29



/opt/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [60]:
# save the model
import joblib

# store the trained model
joblib.dump(model, "../model/svm_sentiment_model.pkl")

# Save the vectorizer (important for preprocessing new text consistently)
# joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['../model/svm_sentiment_model.pkl']

In [61]:
#load the trained model
load_model = joblib.load("../model/svm_sentiment_model.pkl")
# Predict sentiment and test model
new_review = "this film is boring but it is ok in somewhere"
new_processed = preprocess_data(new_review)
X_new = vectorizer.transform([new_processed])
result = load_model.predict(X_new) # -1 = negative, 1 = positive, 0 = neutral
if result == 1:
    print("Positive")
elif result == 0:
    print("Neural")
else:
    print("Negative")

Negative
